In [1]:
import pandas as pd 

df = pd.read_csv('phil_nlp.csv')

In [2]:
df.sample(5)

,title,author,school,sentence_spacy,sentence_str,sentence_length,sentence_lowered,tokenized_txt,lemmatized_str,sentence
210901,Difference And Repetition,Deleuze,Continental,It is the excess in the Idea which explains th...,It is the excess in the Idea which explains th...,288,it is the excess in the idea which explains th...,"['it', 'is', 'the', 'excess', 'in', 'the', 'id...",-PRON- be the excess in the Idea which explai...,It is the excess in the Idea which explains th...
138981,Philosophical Investigations,Wittgenstein,Analytic,so far it is not a move in the languagegame.,so far it is not a move in the languagegame.,44,so far it is not a move in the languagegame.,"['so', 'far', 'it', 'is', 'not', 'move', 'in',...",so far -PRON- be not a move in the languagega...,so far it is not a move in the languagegame.
244248,Off The Beaten Track,Heidegger,Phenomenology,"As the being that it is, consciousness is phen...","As the being that it is, consciousness is phen...",64,"as the being that it is, consciousness is phen...","['as', 'the', 'being', 'that', 'it', 'is', 'co...","as the being that -PRON- be , consciousness b...","As the being that it is, consciousness is phen..."
198176,The Order Of Things,Foucault,Continental,It is not a matter of progressive hierarchizat...,It is not a matter of progressive hierarchizat...,131,it is not a matter of progressive hierarchizat...,"['it', 'is', 'not', 'matter', 'of', 'progressi...",-PRON- be not a matter of progressive hierarc...,It is not a matter of progressive hierarchizat...
338456,Thus Spake Zarathustra,Nietzsche,Nietzsche,there time was my sole contemporary.,there time was my sole contemporary.,36,there time was my sole contemporary.,"['there', 'time', 'was', 'my', 'sole', 'contem...",there time be -PRON- sole contemporary .,there time was my sole contemporary.


In [3]:
len(df['school'].unique())

12

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
CountVectorizer?

Init signature:
CountVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    analyzer='word',
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.int64'>,
)
Docstring:     
Convert a collection of text documents to a matrix of token counts

This implementation produces a sparse representation of the counts using
scipy.sparse.csr_matrix.

If you do not provide an a-priori dictionary and you do not use an analyzer
that does some kind of feature selection then the number of features will
be equal to the vocabulary size found by analyzing the data.

Read more in the :ref:`User Guide <text_feature_extraction>`.

Parameters
----------
input : string {'filename', 'file', 'content'}, default='content'
    If 'filename', the se

In [6]:
cv = CountVectorizer(min_df=5, stop_words='english')

In [8]:
dtm = cv.fit_transform(df['sentence_str'])

In [9]:
dtm

<500x148 sparse matrix of type '<class 'numpy.int64'>'
	with 1727 stored elements in Compressed Sparse Row format>

In [10]:
text_dict = {}

for title in df['title'].unique():
    text_dict[title] = df[df['title']==title]['sentence_str'].str.cat(sep=' . ')



In [25]:
book_df = pd.DataFrame()
book_df['title'] = list(text_dict.keys())
book_df['text'] = list(text_dict.values())

In [26]:
book_df.head()

,title,text
0,Plato - Complete Works,"What's new, Socrates, to make you leave your ..."
1,Aristotle - Complete Works,When things have only a name in common and th...
2,Second Treatise On Government,"Reader, thou hast here the beginning and end o..."
3,Essay Concerning Human Understanding,I have put into thy hands what has been the di...
4,A Treatise Of Human Nature,My design in the present work is sufficiently ...


In [27]:
len(book_df)

56

In [28]:
dtm = cv.fit_transform(book_df['text'])

In [30]:
from sklearn.decomposition import LatentDirichletAllocation

In [31]:
LDA = LatentDirichletAllocation(n_components=12, random_state=17)

In [32]:
LDA.fit(dtm)

LatentDirichletAllocation(n_components=12, random_state=17)

In [33]:
len(cv.get_feature_names())

17887

In [35]:
len(LDA.components_)

12

In [38]:
topic_1 = LDA.components_[0]

top_ten_words = topic_1.argsort()[-10:]

for index in top_ten_words:
    print(cv.get_feature_names()[index])

beings
possible
sense
knowledge
object
way
time
consciousness
experience
world


In [41]:
for topic in range(0, 12):
    topic = LDA.components_[topic]
    top_ten_words = topic.argsort()[-10:]
    print(f'TOPIC {topic}')
    print('======================')
    for index in top_ten_words:
        print(cv.get_feature_names()[index])
    print(f'/n')

TOPIC [31.49752195 21.65078973 14.03591987 ...  0.08333975  0.08333745
  0.08333642]
beings
possible
sense
knowledge
object
way
time
consciousness
experience
world
/n
TOPIC [ 0.08333366 13.14915217 13.17400832 ...  1.78184933 37.84207282
 10.16989987]
certain
fact
time
world
language
case
sense
does
true
say
/n
TOPIC [ 2.5106953   0.60721686 11.33310865 ...  0.08333359  0.08333656
  0.08333771]
work
means
social
time
power
capital
form
production
value
labour
/n
TOPIC [0.08333442 7.97035464 9.53506453 ... 0.08333397 0.08333387 0.08333365]
question
does
da
state
care
present
sein
time
world
dasein
/n
TOPIC [2.16219238 8.92147138 3.2339634  ... 0.0833337  0.08333375 0.08333356]
body
like
case
time
animals
good
does
thing
man
things
/n
TOPIC [29.85989767 51.52467794 54.19769864 ...  0.08333452  0.08333649
  0.08333616]
nature
body
life
world
reason
truth
order
man
madness
god
/n
TOPIC [15.66730815  5.14618686  6.14702339 ...  0.08333423  0.08333486
  0.08333349]
reason
body
man
nature
god